#### Product recommendations in natural language using RAG and LLM
 - Embed user query using same embedding model as the vector database.(Sentence-Transformers)
 - Retrieve similar products using similarity search by query or by embeddings. (ChromaDB)
 - Generate a Natural Language Response using an LLM.
    - Open source models: LLaMA-2/Mistral/Gemma/Phi-2
    - ChatOpen AI  gpt-4

#### Load Pretrained Embedding Model

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Define the storage path
PERSIST_DIRECTORY = "chromadb_vectorstore"

# Initialize ChromaDB and OpenAI embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIRECTORY)  # Pass embeddings to Chroma

/var/folders/y4/z3s1vk012bv7p_188zpx6w3c0000gn/T/ipykernel_60615/2398591998.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/var/folders/y4/z3s1vk012bv7p_188zpx6w3c0000gn/T/ipykernel_60615/2398591998.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(em

### Retrieve similar products

In [19]:
### Retrieve Similar Products
def retrieve_similar_products(query, top_k=5):
    """Retrieve the top-K similar products based on the query."""
    results = vectorstore.similarity_search(query, k=top_k)  # Get top 5 matches
    
    recommended_products = []
    for doc in results:
        recommended_products.append({
            "product_description": doc.page_content,  # Retrieved text
            "product_image": doc.metadata.get("image_url", "No image available"),  # Image link
        })
    
    return recommended_products

# Example user query
query = "Find me Men's Sneaker in black color"
recommended_products = retrieve_similar_products(query)

In [20]:
# Print the recommended products
for i, result in enumerate(recommended_products):
    print(f"Result {i+1}:")
    print(f"Product Description: {result['product_description']}")
    print(f"Product Image: {result['product_image']}")
    print("---")

Result 1:
Product Description: men ' s sneakers in black and grey with red stripes Given Product description: , Care Instructions: Allow your pair of shoes to air and de-odorize at regular basis; Using a Shoe-horn to wear your shoes will avoid damage to the back of your shoes; Use Shoe bags to prevent any stains or mildew., brand: Amazon Brand - Symbol, weight: , color: Black/Blue, height: 1176.0, width: 2560.0, model year: , shape: , style: AZ-SH-05D_Black/Blue_11, material: Canvas, product_type: SHOES
Product Image: No image available
---
Result 2:
Product Description: a black sneaker with white soles Given Product description: , Designed in Europe - please refer to size chart for specific measurements to achieve the perfect fit, brand: find., weight: , color: Black (Black), height: 585.0, width: 1530.0, model year: 2017.0, shape: , style: Men's Retro Trainer Sneakers, material: , product_type: SHOES
Product Image: No image available
---
Result 3:
Product Description: a picture of a 

#### Generate Natual Language Response using LLM

In [ ]:
!pip install python-dotenv
!pip install openai

In [15]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if the API key is loaded correctly
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

In [ ]:
from langchain.chat_models import ChatOpenAI

# Load an LLM (GPT-4 for best responses, or use an open-source model)
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

def generate_natural_language_response(query, products):
    """Generate a response based on retrieved products using LLM."""
    prompt = f"""
    A customer is looking for a product based on this query: "{query}"
    Here are the recommended products:
    {products}
    
    Generate a natural language response listing the products in a friendly tone.
    """

    response = llm.predict(prompt)
    return response

# Generate response
response_text = generate_natural_language_response(query, recommended_products)
print(response_text)

Sure, I found a few options that you might like:

1. The first one is a pair of men's sneakers by Amazon Brand - Symbol. They are black and grey with red stripes. They are made of canvas and come with care instructions to help them last longer. Unfortunately, there's no image available for this product.

2. Next, we have a black sneaker with white soles by the brand 'find'. They were designed in Europe in 2017 and come with a size chart to ensure a perfect fit. Sadly, there's no image available for these shoes.

3. The third option is a black leather sneaker by Amazon Brand - Inkast Denim Co. The color is listed as D.GREY. They also come with care instructions. However, there's no available image.

4. Lastly, we have two similar options by CARE OF by PUMA. They are white men's sneakers, engineered for comfort with an extra-thick foam cushioned insole. Made of leather, they were released in 2019. There are no images available for these shoes.

I hope one of these options will suit your 

#### Todo: 
    - Fine tune LLaMA-2 using LoRA and QLoRA?
    - Inference and Accuracy
    - Compare different open-source LLMs.